In [83]:
import math
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image


### Código para calcular la distancia geométrica entre un pixel y uno de sus vecinos.

In [70]:
def geoDist(x,y):
    
    (s,t) = x
    (m,n) = y
    
    numD = (m-s)**2 + (n-t)**2
    denD = 2*(10**2)
    
    return math.exp(((-1)*numD)/denD)

### Código para calcular la diferencia de intensidad entre un pixel y uno de sus vecinos.

In [71]:
def intDif(gx,gy):
    
    numI = (gx-gy)**2
    denI = 2*(6.7**2)
    
    return math.exp(((-1)*numI)/denI)

### Código para calcular la similitud entre un pixel y uno de sus vecinos.

In [72]:
def simil(img,x,y):
    
    (s,t) = x
    (m,n) = y
    
    gx = img[s,t]
    gy = img[m,n]
    
    return geoDist(x,y)*intDif(gx,gy)

### Código para calcular la similitud local de cada pixel en una matriz.

In [73]:
def imgSimil(img):
    
    imgS = img.copy()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 1 and s < img.shape[0]-2 and t > 1 and t < img.shape[1]-2:
                nx = []
                for p in posiciones:
                    (i,j) = p
                    if p!=(0,0):
                        nx.append((s+i,t+j))
                dsetaX = 0
                for y in nx:
                    dsetaX += simil(img,(s,t),y)
                imgS[s,t] = dsetaX
            else:
                imgS[s,t] = 0
    
    return imgS

In [74]:
def imgLocalSimil(img, imgS):
    
    imgLS = img.copy()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 3 and s < img.shape[0]-4 and t > 3 and t < img.shape[1]-4:
                nx = []
                for p in posiciones:
                    (i,j) = p
                    if p!=(0,0):
                        nx.append((s+i,t+j))
                dsetaY = 0
                for y in nx:
                    (m,n) = y
                    dsetaY += imgS[m,n]
                dsetaX = imgS[s,t]
                medDsetaX = dsetaX / (dsetaY/24)
                if medDsetaX > 2.5:
                    imgLS[s,t] = 1
                else:
                    imgLS[s,t] = medDsetaX/2.5
            else:
                imgLS[s,t] = 0
    
    return imgLS

### Código para calcular la región designada de cada pixel.

In [75]:
def pixReg(img,imgLS):
    
    w1 = 2
    w2 = 4
    tSigma = 0.12
    
    flatReg = list()
    compReg = list()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 3 and s < img.shape[0]-4 and t > 3 and t < img.shape[1]-4:
                nx = []
                for p in posiciones:
                    (i,j) = p
                    if p!=(0,0):
                        nx.append((s+i,t+j))
                a = 0
                b = 0
                for y in nx:
                    (m,n) = y
                    a += imgLS[m,n]**w1
                    b += imgLS[m,n]**w2
                muX = 0
                for y in nx:
                    (m,n) = y
                    muX = ((imgLS[m,n]**w1)*img[m,n])/a
                sigmaX = 0
                for y in nx:
                    (m,n) = y
                    sigmaX = ((imgLS[m,n]**w2)*((img[m,n]-muX)**2))/b
    
                if sigmaX <= tSigma:
                    flatReg.append((s,t))
                else:
                    compReg.append((s,t))
    
    return (flatReg,compReg)

### Código para calcular la naturaleza de un pixel.

In [76]:
def pixNat(img,imgLS):
    
    noise = list()
    clean = list()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if s > 3 and s < img.shape[0]-4 and t > 3 and t < img.shape[1]-4:
                nx = []
                for p in posiciones:
                    (i,j) = p
                    if p!=(0,0):
                        nx.append((s+i,t+j))
                iX = img[s,t]
                yLS = list()
                for y in nx:
                    (m,n) = y
                    yLS.append(imgLS[m,n])
                iY = max(yLS)
                
                if iX - iY > 5:
                    noise.append((s,t))
                else:
                    clean.append((s,t))
    
    return (noise,clean)

### Código para calcular el nivel de ruido en la zona.

In [77]:
def noiseLvl(img,regiones,naturaleza):
    
    (flatReg,compReg) = regiones
    (noise,clean) = naturaleza
    
    sigma = 0
    
    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))

    for x in range(10):
        (s,t) = flatReg[x]
        nx = []
        for p in posiciones:
            (i,j) = p
            if p!=(0,0):
                nx.append((s+i,t+j))
        if flatReg[x] in noise:
            n = 1
            c = 0
        elif flatReg[x] in clean:
            n = 0
            c = 1
        for y in nx:
            if y in noise:
                n += 1
            elif y in clean:
                c += 1
        sigma += n / (c+n)
        
    sigma = sigma/10
    
    thetaF = (-0.12)*(sigma**3) + 0.07*(sigma**2) + 0.75*sigma + 0.19
    thetaC = 0.31*(sigma**3) + 0.63*(sigma**2) + 0.52*sigma + 0.03
    
    
    return (thetaF,thetaC)

### Código para catalogar de forma final los píxeles de la imagen.

In [78]:
def filtrado(img,imgLS):
    regiones = pixReg(img,imgLS)
    naturaleza = pixNat(img,imgLS)
    (thetaF,thetaC) = noiseLvl(img,regiones,naturaleza)
    (flatR,compR) = regiones
    pixelesR = list()
    pixelesL = list()

    for p in flatR:
        (s,t) = p
        lsp = imgLS[s,t]
        if lsp <= thetaF:
            pixelesR.append(p)
        else:
            pixelesL.append(p)
    
    for p in compR:
        (s,t) = p
        lsp = imgLS[s,t]
        if lsp <= thetaC:
            pixelesR.append(p)
        else:
            pixelesL.append(p)
    
    medF = cv.medianBlur(img,5)
    gaussF = cv.GaussianBlur(medF,(5,5),0)

    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            if (s == 0 or t == 0 or s == img.shape[0]-1 or t == img.shape[1]-1) and ((s,t) in pixelesR):
                valor = img[s,t] - gaussF[s,t]
                valorAb = abs(valor)
                if valorAb <= 15:
                    pixelesL.append((s,t))
                    pixelesR.remove((s,t))
    
    return (pixelesR,pixelesL)

### Código para calcular el valor de intensidad de un pixel ruidoso filtrado.

In [79]:
def valFinal(img,imgLS,ruidosos):
    
    resimg = img.copy()
    rLimpiados = img.copy()

    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    for r in ruidosos:
        (s,t) = r
        if s > 3 and s < img.shape[0]-4 and t > 3 and t < img.shape[1]-4:
            nx = []
            for p in posiciones:
                (i,j) = p
                if p!=(0,0):
                    nx.append((s+i,t+j))
            numG = 0
            denG = 0
            for y in nx:
                (m,n) = y
                lY = imgLS[m,n]
                if lY == 0:
                    lY = 0.001
                numG += geoDist((s,t),y)*(lY**2)*img[m,n]
                denG += geoDist((s,t),y)*(lY**2)
            calculo = numG/denG
            rLimpiados[s,t] = calculo
    
    for s in range(img.shape[0]):
        for t in range(img.shape[1]):
            valorOr = img[s,t]
            valorLim = rLimpiados[s,t]
            if valorOr != valorLim:
                resimg[s,t] = valorLim
                
    return resimg

### Código para calcular el PSNR de dos imagenes.

In [80]:
def psnr(imgO,imgR):
    
    mse = np.mean((imgO-imgR)**2)
    if mse == 0:
        return 100
    
    return 10*np.log10((255**2)/mse)

### Código de limpieza de imagenes con RVIN.

In [81]:
def limpiador(direccion):
    
    img = cv.imread(direccion, cv.IMREAD_GRAYSCALE)
    imgO = img.copy()
    imgS = imgSimil(img)
    imgLS = imgLocalSimil(img,imgS)
    (ruidosos,limpios) = filtrado(img,imgLS)
    
    PSNRp = 0
    PSNRim = 1
    
    while PSNRim > PSNRp:
        PSNRp = PSNRim
        imgR = valFinal(imgO,imgLS,ruidosos)
        PSNRim = psnr(imgO,imgR)
        imgO = imgR.copy()
    
    return imgO

In [88]:
imL = limpiador("images/noise50_mandril_gray.png")
cv.imwrite("images/clean50_mandril_gray.png",imL)
Image.open("images/noise50_mandril_gray.png")
Image.open("images/clean50_mandril_gray.png")


OSError: cannot identify image file 'images/clean50_mandril_gray.png'